In [1]:
import pickle
import os
import pandas as pd
import yfinance as yf
import investpy
import math
import numpy as np
from datetime import timedelta
from datetime import date

In [2]:
def fisher(ohlc, period):
    def __round(val):
        if (val > .99):
            return .999
        elif val < -.99:
            return -.999
        return val

    from numpy import log, seterr
    seterr(divide="ignore")
    med = (ohlc["High"] + ohlc["Low"]) / 2
    ndaylow = med.rolling(window=period).min()
    ndayhigh = med.rolling(window=period).max()
    med = [0 if math.isnan(x) else x for x in med]
    ndaylow = [0 if math.isnan(x) else x for x in ndaylow]
    ndayhigh = [0 if math.isnan(x) else x for x in ndayhigh]
    raw = [0] * len(med)
    for i in range(0, len(med)):
        try:
            raw[i] = 2 * ((med[i] - ndaylow[i]) / (ndayhigh[i] - ndaylow[i]) - 0.5)
        except:
            ZeroDivisionError
    value = [0] * len(med)
    value[0] = __round(raw[0] * 0.33)
    for i in range(1, len(med)):
        try:
            value[i] = __round(0.33 * raw[i] + 0.67 * value[i - 1])
        except:
            ZeroDivisionError
    _smooth = [0 if math.isnan(x) else x for x in value]
    fish1 = [0] * len(_smooth)
    for i in range(1, len(_smooth)):
        fish1[i] = ((0.5 * (np.log((1 + _smooth[i]) / (1 - _smooth[i]))))) + (0.5 * fish1[i - 1])
    fish2 = fish1[1:len(fish1)]
    # plt.figure(figsize=(18, 8))
    # plt.plot(ohlc.index, fish1, linewidth=1, label="Fisher_val")
    # plt.legend(loc="upper left")
    # plt.show()
    return fish1
def add_fisher(temp):
    for f_look in range(50, 400, 20):
        temp[f'Fisher{f_look}'] = fisher(temp, f_look)
    return temp
def get_data_investpy( symbol, country, from_date, to_date ):
    find = investpy.search.search_quotes(text=symbol, products=["stocks", "etfs", "indices", "currencies"])
    for f in find:
        #print( f )
        if f.symbol.lower() == symbol.lower() and f.country.lower() == country.lower():
            break
    if f.symbol.lower() != symbol.lower():
        return None
    ret = f.retrieve_historical_data(from_date=from_date, to_date=to_date)
    if ret is None:
        try:
            ret = investpy.get_stock_historical_data(stock=symbol,
                                                     country=country,
                                                     from_date=from_date,
                                                     to_date=to_date)
        except:
            ret = None
    if ret is None:
        try:
            ret = investpy.get_etf_historical_data(etf=symbol,
                                                   country=country,
                                                   from_date=from_date,
                                                   to_date=to_date)
        except:
            ret = None
    if ret is None:
        try:
            ret = investpy.get_index_historical_data(index=symbol,
                                                     country=country,
                                                     from_date=from_date,
                                                     to_date=to_date)
        except:
            ret = None

    if ret is None:
        try:
            ret = investpy.currency_crosses.get_currency_cross_historical_data(currency_cross=symbol,
                                                                               from_date=from_date,
                                                                               to_date=to_date)
        except:
            ret = None
    ret.drop(["Change Pct"], axis=1, inplace=True)
    return ret

def get_data(ticker, api):

    if api == "yfinance":
        temp_og = yf.download(ticker, start = '2007-07-01', end= str(date.today()+timedelta(1)))
        temp_og.reset_index(inplace=True)
        temp_og.drop(['Adj Close'], axis=1, inplace=True)
        temp_og = temp_og.loc[temp_og["Close"]>1]
        temp_og = add_fisher(temp_og)

    if api =="investpy":
        temp_og = get_data_investpy(symbol=ticker, country='india', from_date="01/07/2007",to_date=(date.today()+timedelta(1)).strftime("%d/%m/%Y"))
        temp_og.reset_index(inplace=True)
        temp_og = add_fisher(temp_og)


    return temp_og

def valid_dates(dates_all):
    dates = []
    i = 0
    while True:
        dates.append(dates_all[i])
        if dates_all[i] > pd.to_datetime(date.today()):
            break
        i = i + 1
    return dates


In [6]:
#update indices
tickers = ["TLT"]  #"^NSEI", "GOLDBEES.NS", "^NSEMDCP50", "^IXIC", "TLT"

for ticker in tickers:
    if ticker == "^NSEI":
        number_of_optimization_periods = 1
        recalib_months = 3
        num_strategies = 5
        metric = 'outperformance'
    if ticker == "GOLDBEES.NS":
        number_of_optimization_periods = 2
        recalib_months = 6
        num_strategies = 1
        metric = 'outperformance'
    if ticker == "^NSEMDCP50":
        number_of_optimization_periods = 2
        recalib_months = 12
        num_strategies = 7
        metric = 'maxdrawup_by_maxdrawdown'
    if ticker == "^IXIC":
        number_of_optimization_periods = 2
        recalib_months = 3
        num_strategies = 5
        metric = 'maxdrawup_by_maxdrawdown'
    if ticker == "TLT":
        number_of_optimization_periods = 2
        recalib_months = 6
        num_strategies = 5
        metric = 'rolling_sharpe'

    if number_of_optimization_periods == 1:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_res.pkl', 'rb') as file:
            res_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test = pickle.load(file)
        res_test4 = []
        res_test8 = []
    if number_of_optimization_periods == 2:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_res.pkl', 'rb') as file:
            res_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test = pickle.load(file)
        res_test8 = []
    if number_of_optimization_periods == 3:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_8_All_Strategies.pkl', 'rb') as file:
            res_test8 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_res.pkl', 'rb') as file:
            res_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test = pickle.load(file)

    temp_og = get_data(ticker, "yfinance")
    dates = valid_dates(pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10],
                                      end="2024-06-15", freq=f'3M'))
    datesb = []
    for date_i in range(len(dates) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            datesb.append(dates[date_i + int(24 / 3)])
    datesb.append(date.today())
    weights_supposed_to_be = [None] * (len(datesb)-1)

    with open(f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl', 'rb') as file:
        weights = pickle.load(file)

    if not os.path.exists(f'output/{ticker}'):
        continue

    if number_of_optimization_periods == 1:
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_res.pkl', 'rb') as file:
            res_test_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test_imp = pickle.load(file)
        res_test4_imp = []
        res_test8_imp = []
    if number_of_optimization_periods == 2:
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_res.pkl', 'rb') as file:
            res_test_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test_imp = pickle.load(file)
        res_test8_imp = []
    if number_of_optimization_periods == 3:
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_8_All_Strategies.pkl', 'rb') as file:
            res_test8_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_res.pkl', 'rb') as file:
            res_test_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test_imp = pickle.load(file)

    with open(f'output/{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl', 'rb') as file:
        weights_imp = pickle.load(file)

    # print(weights)

    print("*"*100)
    print(ticker)
    print(f"len2: {len(res_test2)}")
    print(f"len4: {len(res_test4)}")
    print(f"len8: {len(res_test8)}")
    print(f"len_res_test: {len(res_test)}")
    print(f"len_ss_test: {len(ss_test)}")
    print(f"Weights: {len(weights)}")
    print(f"weights_supposed_to_be: {len(weights_supposed_to_be)}")
    print(f"len2_imp: {len(res_test2_imp)}")
    print(f"len4_imp: {len(res_test4_imp)}")
    print(f"len8_imp: {len(res_test8_imp)}")
    print(f"len_res_test_imp: {len(res_test_imp)}")
    print(f"len_ss_test_imp: {len(ss_test_imp)}")
    print(f"Weights_imp: {len(weights_imp)}")
    print("*"*100)

    # if number_of_optimization_periods == 1:
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test2+res_test2_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_res.pkl', 'wb') as file:
    #         pickle.dump((res_test+res_test_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_ss.pkl', 'wb') as file:
    #         pickle.dump((ss_test+ss_test_imp), file)
    #     res_test4 = []
    #     res_test8 = []
    # if number_of_optimization_periods == 2:
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test2+res_test2_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test4+res_test4_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_res.pkl', 'wb') as file:
    #         pickle.dump((res_test+res_test_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_ss.pkl', 'wb') as file:
    #         pickle.dump((ss_test+ss_test_imp),file)
    #     res_test8 = []
    # if number_of_optimization_periods == 3:
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test2+res_test2_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test4+res_test4_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_8_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test8+res_test8_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_res.pkl', 'wb') as file:
    #         pickle.dump((res_test+res_test_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_ss.pkl', 'wb') as file:
    #         pickle.dump((ss_test+ss_test_imp),file)

    # print(len(weights))
    # print(len(weights_supposed_to_be))
    # if len(weights)<len(weights_supposed_to_be):
    #     # print("here")

    with open(f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl', 'wb') as file:
        pickle.dump((weights+[weights_imp[0][1]]), file)


[*********************100%***********************]  1 of 1 completed
****************************************************************************************************
TLT
len2: 50
len4: 50
len8: 0
len_res_test: 50
len_ss_test: 50
Weights: 24
weights_supposed_to_be: 24
len2_imp: 1
len4_imp: 1
len8_imp: 0
len_res_test_imp: 1
len_ss_test_imp: 1
Weights_imp: 1
****************************************************************************************************


In [3]:

constituent_alpha_params = {'ROST': {'ticker_yfinance': 'ROST',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'rolling_sharpe'},
        'MNST': {'ticker_yfinance': 'MNST',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'CMCSA': {'ticker_yfinance': 'CMCSA',
                  'number_of_optimization_periods': 3,
                  'recalib_months': 12,
                  'num_strategies': 7,
                  'metric': 'rolling_cagr'},
        'KLAC': {'ticker_yfinance': 'KLAC',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_sortino'},
        'NXPI': {'ticker_yfinance': 'NXPI',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_sortino'},
        'SHPG': {'ticker_yfinance': 'SHPG',
                 'number_of_optimization_periods': 0,
                 'recalib_months': 0,
                 'num_strategies': 0,
                 'metric': ''},
        'XLNX': {'ticker_yfinance': 'XLNX',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 3,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'ALGN': {'ticker_yfinance': 'ALGN',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'MRVL': {'ticker_yfinance': 'MRVL',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'ISRG': {'ticker_yfinance': 'ISRG',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 1,
                 'metric': 'rolling_sharpe'},
        'MAT': {'ticker_yfinance': 'MAT',
                'number_of_optimization_periods': 1,
                'recalib_months': 3,
                'num_strategies': 7,
                'metric': 'maxdrawup_by_maxdrawdown'},
        'OKTA': {'ticker_yfinance': 'OKTA',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'AVGO': {'ticker_yfinance': 'AVGO',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'rolling_sharpe'},
        'DXCM': {'ticker_yfinance': 'DXCM',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'AMD': {'ticker_yfinance': 'AMD',
                'number_of_optimization_periods': 3,
                'recalib_months': 6,
                'num_strategies': 3,
                'metric': 'rolling_sortino'},
        'DOCU': {'ticker_yfinance': 'DOCU',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 7,
                 'metric': 'rolling_cagr'},
        'INTC': {'ticker_yfinance': 'INTC',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 3,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'UAL': {'ticker_yfinance': 'UAL',
                'number_of_optimization_periods': 1,
                'recalib_months': 6,
                'num_strategies': 7,
                'metric': 'rolling_cagr'},
        'KDP': {'ticker_yfinance': 'KDP',
                'number_of_optimization_periods': 3,
                'recalib_months': 6,
                'num_strategies': 5,
                'metric': 'maxdrawup_by_maxdrawdown'},
        'WBA': {'ticker_yfinance': 'WBA',
                'number_of_optimization_periods': 3,
                'recalib_months': 3,
                'num_strategies': 7,
                'metric': 'rolling_cagr'},
        'CSCO': {'ticker_yfinance': 'CSCO',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'rolling_sortino'},
        'SIRI': {'ticker_yfinance': 'SIRI',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 1,
                 'metric': 'outperformance'},
        'LRCX': {'ticker_yfinance': 'LRCX',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'rolling_sharpe'},
        'GILD': {'ticker_yfinance': 'GILD',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'ADP': {'ticker_yfinance': 'ADP',
                'number_of_optimization_periods': 3,
                'recalib_months': 6,
                'num_strategies': 5,
                'metric': 'outperformance'},
        'NLOK': {'ticker_yfinance': 'NLOK',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'ADSK': {'ticker_yfinance': 'ADSK',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 1,
                 'metric': 'rolling_sortino'},
        'AMZN': {'ticker_yfinance': 'AMZN',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 3,
                 'metric': 'outperformance'},
        'QRTEA': {'ticker_yfinance': 'QRTEA',
                  'number_of_optimization_periods': 1,
                  'recalib_months': 12,
                  'num_strategies': 7,
                  'metric': 'outperformance'},
        'REGN': {'ticker_yfinance': 'REGN',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'LBTYA': {'ticker_yfinance': 'LBTYA',
                  'number_of_optimization_periods': 1,
                  'recalib_months': 12,
                  'num_strategies': 5,
                  'metric': 'outperformance'},
        'TMUS': {'ticker_yfinance': 'TMUS',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 6,
                 'num_strategies': 3,
                 'metric': 'rolling_sortino'},
        'LULU': {'ticker_yfinance': 'LULU',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'SGEN': {'ticker_yfinance': 'SGEN',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'rolling_sharpe'},
        'MDLZ': {'ticker_yfinance': 'MDLZ',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 1,
                 'metric': 'rolling_sortino'},
        'INCY': {'ticker_yfinance': 'INCY',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 3,
                 'metric': 'rolling_sortino'},
        'TCOM': {'ticker_yfinance': 'TCOM',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'STX': {'ticker_yfinance': 'STX',
                'number_of_optimization_periods': 1,
                'recalib_months': 12,
                'num_strategies': 7,
                'metric': 'rolling_sharpe'},
        'CDNS': {'ticker_yfinance': 'CDNS',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 1,
                 'metric': 'rolling_sharpe'},
        'NTAP': {'ticker_yfinance': 'NTAP',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'HAS': {'ticker_yfinance': 'HAS',
                'number_of_optimization_periods': 3,
                'recalib_months': 12,
                'num_strategies': 5,
                'metric': 'rolling_sharpe'},
        'CHTR': {'ticker_yfinance': 'CHTR',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'outperformance'},
        'ILMN': {'ticker_yfinance': 'ILMN',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'rolling_sortino'},
        'SBUX': {'ticker_yfinance': 'SBUX',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'rolling_sortino'},
        'PYPL': {'ticker_yfinance': 'PYPL',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'rolling_sharpe'},
        'EBAY': {'ticker_yfinance': 'EBAY',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'AMGN': {'ticker_yfinance': 'AMGN',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'TEAM': {'ticker_yfinance': 'TEAM',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'MCHP': {'ticker_yfinance': 'MCHP',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'rolling_sortino'},
        'BIDU': {'ticker_yfinance': 'BIDU',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 7,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'NCLH': {'ticker_yfinance': 'NCLH',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'rolling_sharpe'},
        'EA': {'ticker_yfinance': 'EA',
               'number_of_optimization_periods': 3,
               'recalib_months': 6,
               'num_strategies': 1,
               'metric': 'rolling_sortino'},
        'XEL': {'ticker_yfinance': 'XEL',
                'number_of_optimization_periods': 1,
                'recalib_months': 3,
                'num_strategies': 7,
                'metric': 'rolling_sortino'},
        'CERN': {'ticker_yfinance': 'CERN',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'CDW': {'ticker_yfinance': 'CDW',
                'number_of_optimization_periods': 2,
                'recalib_months': 6,
                'num_strategies': 1,
                'metric': 'rolling_sortino'},
        'AMAT': {'ticker_yfinance': 'AMAT',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'outperformance'},
        'CPRT': {'ticker_yfinance': 'CPRT',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'BKNG': {'ticker_yfinance': 'BKNG',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'CTSH': {'ticker_yfinance': 'CTSH',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 1,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'AEP': {'ticker_yfinance': 'AEP',
                'number_of_optimization_periods': 2,
                'recalib_months': 12,
                'num_strategies': 7,
                'metric': 'maxdrawup_by_maxdrawdown'},
        'CHKP': {'ticker_yfinance': 'CHKP',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_sortino'},
        'PEP': {'ticker_yfinance': 'PEP',
                'number_of_optimization_periods': 3,
                'recalib_months': 6,
                'num_strategies': 5,
                'metric': 'rolling_sortino'},
        'FB': {'ticker_yfinance': 'FB',
               'number_of_optimization_periods': 1,
               'recalib_months': 12,
               'num_strategies': 3,
               'metric': 'rolling_cagr'},
        'JD': {'ticker_yfinance': 'JD',
               'number_of_optimization_periods': 2,
               'recalib_months': 6,
               'num_strategies': 7,
               'metric': 'rolling_sortino'},
        'ANSS': {'ticker_yfinance': 'ANSS',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'rolling_sharpe'},
        'VTRS': {'ticker_yfinance': 'VTRS',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 3,
                 'metric': 'rolling_sortino'},
        'INTU': {'ticker_yfinance': 'INTU',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'outperformance'},
        'LILA': {'ticker_yfinance': 'LILA',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'rolling_cagr'},
        'CSGP': {'ticker_yfinance': 'CSGP',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 7,
                 'metric': 'rolling_sharpe'},
        'NVDA': {'ticker_yfinance': 'NVDA',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'GOOGL': {'ticker_yfinance': 'GOOGL',
                  'number_of_optimization_periods': 3,
                  'recalib_months': 6,
                  'num_strategies': 5,
                  'metric': 'outperformance'},
        'VOD': {'ticker_yfinance': 'VOD',
                'number_of_optimization_periods': 3,
                'recalib_months': 12,
                'num_strategies': 7,
                'metric': 'maxdrawup_by_maxdrawdown'},
        'NFLX': {'ticker_yfinance': 'NFLX',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'JBHT': {'ticker_yfinance': 'JBHT',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 1,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'XRAY': {'ticker_yfinance': 'XRAY',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'rolling_sharpe'},
        'DLTR': {'ticker_yfinance': 'DLTR',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'VRTX': {'ticker_yfinance': 'VRTX',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'COST': {'ticker_yfinance': 'COST',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'outperformance'},
        'IDXX': {'ticker_yfinance': 'IDXX',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'rolling_cagr'},
        'TTWO': {'ticker_yfinance': 'TTWO',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'rolling_sortino'},
        'FISV': {'ticker_yfinance': 'FISV',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 1,
                 'metric': 'rolling_sortino'},
        'AKAM': {'ticker_yfinance': 'AKAM',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'rolling_sharpe'},
        'ADBE': {'ticker_yfinance': 'ADBE',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'rolling_sharpe'},
        'NTES': {'ticker_yfinance': 'NTES',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'BIIB': {'ticker_yfinance': 'BIIB',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'SWKS': {'ticker_yfinance': 'SWKS',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 1,
                 'metric': 'rolling_sharpe'},
        'SNPS': {'ticker_yfinance': 'SNPS',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'rolling_cagr'},
        'AAL': {'ticker_yfinance': 'AAL',
                'number_of_optimization_periods': 1,
                'recalib_months': 6,
                'num_strategies': 5,
                'metric': 'maxdrawup_by_maxdrawdown'},
        'EXC': {'ticker_yfinance': 'EXC',
                'number_of_optimization_periods': 3,
                'recalib_months': 3,
                'num_strategies': 1,
                'metric': 'rolling_cagr'},
        'DISH': {'ticker_yfinance': 'DISH',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'MU': {'ticker_yfinance': 'MU',
               'number_of_optimization_periods': 1,
               'recalib_months': 12,
               'num_strategies': 3,
               'metric': 'maxdrawup_by_maxdrawdown'},
        'VRSN': {'ticker_yfinance': 'VRSN',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 1,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'QCOM': {'ticker_yfinance': 'QCOM',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 3,
                 'metric': 'outperformance'},
        'TSCO': {'ticker_yfinance': 'TSCO',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'MELI': {'ticker_yfinance': 'MELI',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_sortino'},
        'HOLX': {'ticker_yfinance': 'HOLX',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 3,
                 'metric': 'rolling_sortino'},
        'WYNN': {'ticker_yfinance': 'WYNN',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 3,
                 'metric': 'rolling_sortino'},
        'EXPE': {'ticker_yfinance': 'EXPE',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'BMRN': {'ticker_yfinance': 'BMRN',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'FAST': {'ticker_yfinance': 'FAST',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'outperformance'},
        'ASML': {'ticker_yfinance': 'ASML',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 1,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'TSLA': {'ticker_yfinance': 'TSLA',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'KHC': {'ticker_yfinance': 'KHC',
                'number_of_optimization_periods': 3,
                'recalib_months': 12,
                'num_strategies': 3,
                'metric': 'rolling_cagr'},
        'MSFT': {'ticker_yfinance': 'MSFT',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'ORLY': {'ticker_yfinance': 'ORLY',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'outperformance'},
        'PAYX': {'ticker_yfinance': 'PAYX',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'rolling_sortino'},
        'CTXS': {'ticker_yfinance': 'CTXS',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'PCAR': {'ticker_yfinance': 'PCAR',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 3,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'ULTA': {'ticker_yfinance': 'ULTA',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'CSX': {'ticker_yfinance': 'CSX',
                'number_of_optimization_periods': 1,
                'recalib_months': 6,
                'num_strategies': 5,
                'metric': 'rolling_sharpe'},
        'DISCA': {'ticker_yfinance': 'DISCA',
                  'number_of_optimization_periods': 2,
                  'recalib_months': 12,
                  'num_strategies': 1,
                  'metric': 'rolling_sharpe'},
        'WDC': {'ticker_yfinance': 'WDC',
                'number_of_optimization_periods': 3,
                'recalib_months': 3,
                'num_strategies': 7,
                'metric': 'rolling_cagr'},
        'ADI': {'ticker_yfinance': 'ADI',
                'number_of_optimization_periods': 3,
                'recalib_months': 6,
                'num_strategies': 1,
                'metric': 'rolling_sharpe'},
        'HSIC': {'ticker_yfinance': 'HSIC',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'TXN': {'ticker_yfinance': 'TXN',
                'number_of_optimization_periods': 3,
                'recalib_months': 3,
                'num_strategies': 7,
                'metric': 'rolling_sharpe'},
        'ATVI': {'ticker_yfinance': 'ATVI',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'rolling_cagr'},
        'AAPL': {'ticker_yfinance': 'AAPL',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 3,
                 'metric': 'rolling_cagr'},
        'CTAS': {'ticker_yfinance': 'CTAS',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 3,
                 'num_strategies': 3,
                 'metric': 'outperformance'},
        'SPLK': {'ticker_yfinance': 'SPLK',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 6,
                 'num_strategies': 5,
                 'metric': 'rolling_cagr'},
        'HON': {'ticker_yfinance': 'HON',
                'number_of_optimization_periods': 3,
                'recalib_months': 3,
                'num_strategies': 7,
                'metric': 'maxdrawup_by_maxdrawdown'},
        'VRSK': {'ticker_yfinance': 'VRSK',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'PDD': {'ticker_yfinance': 'PDD',
                'number_of_optimization_periods': 2,
                'recalib_months': 12,
                'num_strategies': 5,
                'metric': 'rolling_sharpe'},
        'TRIP': {'ticker_yfinance': 'TRIP',
                 'number_of_optimization_periods': 3,
                 'recalib_months': 12,
                 'num_strategies': 7,
                 'metric': 'outperformance'},
        'MTCH': {'ticker_yfinance': 'MTCH',
                 'number_of_optimization_periods': 2,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'maxdrawup_by_maxdrawdown'},
        'WDAY': {'ticker_yfinance': 'WDAY',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 6,
                 'num_strategies': 7,
                 'metric': 'rolling_sharpe'},
        'SBAC': {'ticker_yfinance': 'SBAC',
                 'number_of_optimization_periods': 1,
                 'recalib_months': 12,
                 'num_strategies': 5,
                 'metric': 'outperformance'},
        'TAMdv': {"ticker_yfinance": "TATAMTRDVR.NS", "number_of_optimization_periods": 1,
                  "recalib_months": 12, "num_strategies": 5, "metric": 'rolling_sharpe'},
        'SBI': {"ticker_yfinance": "SBIN.NS", "number_of_optimization_periods": 3, "recalib_months": 3,
                "num_strategies": 7, "metric": 'rolling_sortino'},
        'NEST': {"ticker_yfinance": "NESTLEIND.NS", "number_of_optimization_periods": 3,
                 "recalib_months": 6, "num_strategies": 5, "metric": 'rolling_sortino'},
        'INFY': {"ticker_yfinance": "INFY.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'outperformance'},
        'TCS': {"ticker_yfinance": "TCS.NS", "number_of_optimization_periods": 3, "recalib_months": 3,
                "num_strategies": 7, "metric": 'outperformance'},
        'COAL': {"ticker_yfinance": "COALINDIA.NS", "number_of_optimization_periods": 3,
                 "recalib_months": 12, "num_strategies": 7, "metric": 'rolling_sortino'},
        'HCLT': {"ticker_yfinance": "HCLTECH.NS", "number_of_optimization_periods": 2, "recalib_months": 6,
                 "num_strategies": 5, "metric": 'rolling_cagr'},
        'NTPC': {"ticker_yfinance": "NTPC.NS", "number_of_optimization_periods": 3, "recalib_months": 12,
                 "num_strategies": 7, "metric": 'rolling_sortino'},
        'ICBK': {"ticker_yfinance": "ICICIBANK.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 6, "num_strategies": 7, "metric": 'rolling_sortino'},
        'LART': {"ticker_yfinance": "LT.NS", "number_of_optimization_periods": 3, "recalib_months": 12,
                 "num_strategies": 7, "metric": 'rolling_sharpe'},
        'HDBK': {"ticker_yfinance": "HDFCBANK.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 12, "num_strategies": 5, "metric": 'rolling_cagr'},
        'TAMO': {"ticker_yfinance": "TATAMOTORS.NS", "number_of_optimization_periods": 1,
                 "recalib_months": 12, "num_strategies": 5, "metric": 'outperformance'},
        'TISC': {"ticker_yfinance": "TATASTEEL.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 3, "num_strategies": 1, "metric": 'rolling_sortino'},
        'BAJA': {"ticker_yfinance": "BAJAJ-AUTO.NS", "number_of_optimization_periods": 1,
                 "recalib_months": 3, "num_strategies": 7, "metric": 'outperformance'},
        'ASPN': {"ticker_yfinance": "ASIANPAINT.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 12, "num_strategies": 5, "metric": 'rolling_sortino'},
        'REDY': {"ticker_yfinance": "DRREDDY.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                 "num_strategies": 7, "metric": 'rolling_cagr'},
        'TEML': {"ticker_yfinance": "TECHM.NS", "number_of_optimization_periods": 1, "recalib_months": 3,
                 "num_strategies": 7, "metric": 'outperformance'},
        'CIPL': {"ticker_yfinance": "CIPLA.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'outperformance'},
        'ULTC': {"ticker_yfinance": "ULTRACEMCO.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 3, "num_strategies": 3, "metric": 'rolling_sharpe'},
        'BJFS': {"ticker_yfinance": "BAJAJFINSV.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 6, "num_strategies": 3, "metric": 'rolling_sortino'},
        'HDFC': {"ticker_yfinance": "HDFC.NS", "number_of_optimization_periods": 2, "recalib_months": 6,
                 "num_strategies": 7, "metric": 'rolling_sharpe'},
        'SUN': {"ticker_yfinance": "SUNPHARMA.NS", "number_of_optimization_periods": 3,
                "recalib_months": 12, "num_strategies": 3, "metric": 'outperformance'},
        'ITC': {"ticker_yfinance": "ITC.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                "num_strategies": 5, "metric": 'rolling_sortino'},
        'WIPR': {"ticker_yfinance": "WIPRO.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                 "num_strategies": 3, "metric": 'rolling_sharpe'},
        'GAIL': {"ticker_yfinance": "GAIL.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 1, "metric": 'rolling_sortino'},
        'VDAN': {"ticker_yfinance": "VEDL.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                 "num_strategies": 1, "metric": 'maxdrawup_by_maxdrawdown'},
        'PGRD': {"ticker_yfinance": "POWERGRID.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 12, "num_strategies": 3, "metric": 'rolling_sortino'},
        'HROM': {"ticker_yfinance": "HEROMOTOCO.NS", "number_of_optimization_periods": 2,
                 "recalib_months": 12, "num_strategies": 5, "metric": 'rolling_sortino'},
        'AXBK': {"ticker_yfinance": "AXISBANK.NS", "number_of_optimization_periods": 1,
                 "recalib_months": 12, "num_strategies": 7, "metric": 'outperformance'},
        'YESB': {"ticker_yfinance": "YESBANK.NS", "number_of_optimization_periods": 3, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'maxdrawup_by_maxdrawdown'},
        'ONGC': {"ticker_yfinance": "ONGC.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                 "num_strategies": 5, "metric": 'maxdrawup_by_maxdrawdown'},
        'HLL': {"ticker_yfinance": "HINDUNILVR.NS", "number_of_optimization_periods": 2,
                "recalib_months": 12,
                "num_strategies": 1, "metric": 'rolling_sharpe'},
        'APSE': {"ticker_yfinance": "ADANIPORTS.NS", "number_of_optimization_periods": 3,
                 "recalib_months": 3,
                 "num_strategies": 5, "metric": 'outperformance'},
        'BRTI': {"ticker_yfinance": "BHARTIARTL.NS", "number_of_optimization_periods": 1,
                 "recalib_months": 12,
                 "num_strategies": 5, "metric": 'maxdrawup_by_maxdrawdown'},
        'VODA': {"ticker_yfinance": "IDEA.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'rolling_sortino'},
        'BFRG': {"ticker_yfinance": "BHARATFORG.NS", "number_of_optimization_periods": 3, "recalib_months": 3,
                 "num_strategies": 7, "metric": 'rolling_sortino'},
        'CUMM': {"ticker_yfinance": "CUMMINSIND.NS", "number_of_optimization_periods": 1, "recalib_months": 3,
                 "num_strategies": 1, "metric": 'outperformance'},
        'CAST': {"ticker_yfinance": "CASTROLIND.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                 "num_strategies": 3, "metric": 'rolling_sortino'},
        'ASOK': {"ticker_yfinance": "ASHOKLEY.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 3, "metric": 'rolling_sharpe'},
        'AUFI': {"ticker_yfinance": "AUBANK.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 7, "metric": 'rolling_cagr'},
        'SRTR': {"ticker_yfinance": "SRTRANSFIN.NS", "number_of_optimization_periods": 1, "recalib_months": 6,
                 "num_strategies": 5, "metric": 'rolling_cagr'},
        'MAXI': {"ticker_yfinance": "MFSL.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                 "num_strategies": 3, "metric": 'maxdrawup_by_maxdrawdown'},
        'BATA': {"ticker_yfinance": "BATAINDIA.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'maxdrawup_by_maxdrawdown'},
        'MINT': {"ticker_yfinance": "MINDTREE.NS", "number_of_optimization_periods": 1, "recalib_months": 6,
                 "num_strategies": 7, "metric": 'maxdrawup_by_maxdrawdown'},
        'COFO': {"ticker_yfinance": "COFORGE.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                 "num_strategies": 7, "metric": 'rolling_cagr'},
        'TVSM': {"ticker_yfinance": "TVSMOTOR.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'rolling_sharpe'},
        'PAGE': {"ticker_yfinance": "PAGEIND.NS", "number_of_optimization_periods": 1, "recalib_months": 3,
                 "num_strategies": 3, "metric": 'maxdrawup_by_maxdrawdown'},
        'CCRI': {"ticker_yfinance": "CONCOR.NS", "number_of_optimization_periods": 1, "recalib_months": 6,
                 "num_strategies": 5, "metric": 'rolling_cagr'},
        'ESCO': {"ticker_yfinance": "ESCORTS.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                 "num_strategies": 7, "metric": 'rolling_cagr'},
        'SRFL': {"ticker_yfinance": "SRF.NS", "number_of_optimization_periods": 2, "recalib_months": 6,
                 "num_strategies": 5, "metric": 'maxdrawup_by_maxdrawdown'},
        'CNBK': {"ticker_yfinance": "CANBK.NS", "number_of_optimization_periods": 3, "recalib_months": 6,
                 "num_strategies": 7, "metric": 'maxdrawup_by_maxdrawdown'},
        'TTPW': {"ticker_yfinance": "TATAPOWER.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'rolling_sharpe'},
        'ZEE': {"ticker_yfinance": "ZEEL.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                "num_strategies": 3, "metric": 'maxdrawup_by_maxdrawdown'},
        'MNFL': {"ticker_yfinance": "MANAPPURAM.NS", "number_of_optimization_periods": 3, "recalib_months": 12,
                 "num_strategies": 3, "metric": 'maxdrawup_by_maxdrawdown'},
        'FED': {"ticker_yfinance": "FEDERALBNK.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                "num_strategies": 7, "metric": 'rolling_sharpe'},
        'GLEN': {"ticker_yfinance": "GLENMARK.NS", "number_of_optimization_periods": 2, "recalib_months": 12,
                 "num_strategies": 7, "metric": 'rolling_cagr'},
        'CHLA': {"ticker_yfinance": "CHOLAFIN.NS", "number_of_optimization_periods": 1, "recalib_months": 3,
                 "num_strategies": 3, "metric": 'maxdrawup_by_maxdrawdown'},
        'AMAR': {"ticker_yfinance": "AMARAJABAT.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 5, "metric": 'outperformance'},
        'APLO': {"ticker_yfinance": "APOLLOTYRE.NS", "number_of_optimization_periods": 1, "recalib_months": 6,
                 "num_strategies": 3, "metric": 'maxdrawup_by_maxdrawdown'},
        'BAJE': {"ticker_yfinance": "BEL.NS", "number_of_optimization_periods": 1, "recalib_months": 6,
                 "num_strategies": 1, "metric": 'rolling_sortino'},
        'SAIL': {"ticker_yfinance": "SAIL.NS", "number_of_optimization_periods": 1, "recalib_months": 3,
                 "num_strategies": 1, "metric": 'rolling_cagr'},
        'MMFS': {"ticker_yfinance": "M&MFIN.NS", "number_of_optimization_periods": 3, "recalib_months": 12,
                 "num_strategies": 7, "metric": 'rolling_cagr'},
        'BLKI': {"ticker_yfinance": "BALKRISIND.NS", "number_of_optimization_periods": 3, "recalib_months": 6,
                 "num_strategies": 5, "metric": 'outperformance'},
        'PWFC': {"ticker_yfinance": "PFC.NS", "number_of_optimization_periods": 2, "recalib_months": 6,
                 "num_strategies": 7, "metric": 'outperformance'},
        'TOPO': {"ticker_yfinance": "TORNTPOWER.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 1, "metric": 'outperformance'},
        'BOB': {"ticker_yfinance": "BANKBARODA.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                "num_strategies": 5, "metric": 'rolling_sortino'},
        'GODR': {"ticker_yfinance": "GODREJPROP.NS", "number_of_optimization_periods": 3, "recalib_months": 12,
                 "num_strategies": 3, "metric": 'rolling_cagr'},
        'LTFH': {"ticker_yfinance": "L&TFH.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 3, "metric": 'rolling_sortino'},
        'INBF': {"ticker_yfinance": "IBULHSGFIN.NS", "number_of_optimization_periods": 1, "recalib_months": 3,
                 "num_strategies": 1, "metric": 'rolling_cagr'},
        'BOI': {"ticker_yfinance": "BANKINDIA.NS", "number_of_optimization_periods": 3, "recalib_months": 3,
                "num_strategies": 7, "metric": 'maxdrawup_by_maxdrawdown'},
        'JNSP': {"ticker_yfinance": "JINDALSTEL.NS", "number_of_optimization_periods": 3, "recalib_months": 6,
                 "num_strategies": 7, "metric": 'rolling_sortino'},
        'IDFB': {"ticker_yfinance": "IDFCFIRSTB.NS", "number_of_optimization_periods": 3, "recalib_months": 3,
                 "num_strategies": 3, "metric": 'rolling_sharpe'},
        'SUTV': {"ticker_yfinance": "SUNTV.NS", "number_of_optimization_periods": 3, "recalib_months": 12,
                 "num_strategies": 1, "metric": 'rolling_cagr'},
        'VOLT': {"ticker_yfinance": "VOLTAS.NS", "number_of_optimization_periods": 1, "recalib_months": 3,
                 "num_strategies": 1, "metric": 'outperformance'},
        'MGAS': {"ticker_yfinance": "MGL.NS", "number_of_optimization_periods": 3, "recalib_months": 3,
                 "num_strategies": 3, "metric": 'rolling_sortino'},
        'RECM': {"ticker_yfinance": "RECLTD.NS", "number_of_optimization_periods": 2, "recalib_months": 3,
                 "num_strategies": 5, "metric": 'rolling_sortino'},
        'GMRI': {"ticker_yfinance": "GMRINFRA.NS", "number_of_optimization_periods": 1, "recalib_months": 6,
                 "num_strategies": 7, "metric": 'outperformance'},
        'BHEL': {"ticker_yfinance": "BHEL.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 1, "metric": 'rolling_sortino'},
        'LICH': {"ticker_yfinance": "LICHSGFIN.NS", "number_of_optimization_periods": 1, "recalib_months": 6,
                 "num_strategies": 7, "metric": 'rolling_sharpe'},
        'EXID': {"ticker_yfinance": "EXIDEIND.NS", "number_of_optimization_periods": 1, "recalib_months": 12,
                 "num_strategies": 1, "metric": 'rolling_sharpe'},
        'TRCE': {"ticker_yfinance": "RAMCOCEM.NS", "number_of_optimization_periods": 2, "recalib_months": 6,
                 "num_strategies": 5, "metric": 'rolling_sharpe'}
    }


for ticker in list(constituent_alpha_params.keys()):

    if ticker not in ['LTFH']:  #, 'CAST', 'COAL', '', 'LBTYA', 'LTFH', 'MSFT', 'TCOM'
        continue

    number_of_optimization_periods = constituent_alpha_params[ticker]["number_of_optimization_periods"]
    recalib_months = constituent_alpha_params[ticker]["recalib_months"]
    num_strategies = constituent_alpha_params[ticker]["num_strategies"]
    metric = constituent_alpha_params[ticker]["metric"]
    if number_of_optimization_periods == 1:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_res.pkl', 'rb') as file:
            res_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test = pickle.load(file)
        res_test4 = []
        res_test8 = []
    if number_of_optimization_periods == 2:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_res.pkl', 'rb') as file:
            res_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test = pickle.load(file)
        res_test8 = []
    if number_of_optimization_periods == 3:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_8_All_Strategies.pkl', 'rb') as file:
            res_test8 = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_res.pkl', 'rb') as file:
            res_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test = pickle.load(file)

    temp_og = get_data(ticker, "investpy")

    dates = valid_dates(pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10],
                                      end="2024-06-15", freq=f'3M'))
    datesb = []
    for date_i in range(len(dates) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            datesb.append(dates[date_i + int(24 / 3)])
    datesb.append(date.today())
    weights_supposed_to_be = [None] * (len(datesb)-1)

    with open(f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl', 'rb') as file:
        weights = pickle.load(file)

    if not os.path.exists(f'output/{ticker}'):
        continue

    if number_of_optimization_periods == 1:
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_res.pkl', 'rb') as file:
            res_test_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test_imp = pickle.load(file)
        res_test4_imp = []
        res_test8_imp = []
    if number_of_optimization_periods == 2:
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_res.pkl', 'rb') as file:
            res_test_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test_imp = pickle.load(file)
        res_test8_imp = []
    if number_of_optimization_periods == 3:
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'rb') as file:
            res_test2_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'rb') as file:
            res_test4_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_TrainYrs_8_All_Strategies.pkl', 'rb') as file:
            res_test8_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_res.pkl', 'rb') as file:
            res_test_imp = pickle.load(file)
        with open(f'output/{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_ss.pkl', 'rb') as file:
            ss_test_imp = pickle.load(file)
    try:
        with open(f'output/{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl', 'rb') as file:
            weights_imp = pickle.load(file)
    except:
        weights_imp = []
    print("*"*100)
    print(ticker)
    print(f"len2: {len(res_test2)}")
    print(f"len4: {len(res_test4)}")
    print(f"len8: {len(res_test8)}")
    print(f"len_res_test: {len(res_test)}")
    print(f"len_ss_test: {len(ss_test)}")
    print(f"Weights: {len(weights)}")
    print(f"weights_supposed_to_be: {len(weights_supposed_to_be)}")
    print(f"len2_imp: {len(res_test2_imp)}")
    print(f"len4_imp: {len(res_test4_imp)}")
    print(f"len8_imp: {len(res_test8_imp)}")
    print(f"len_res_test_imp: {len(res_test_imp)}")
    print(f"len_ss_test_imp: {len(ss_test_imp)}")
    print(f"Weights_imp: {len(weights_imp)}")
    print("*"*100)
    #
    # if number_of_optimization_periods == 1:
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test2+res_test2_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_res.pkl', 'wb') as file:
    #         pickle.dump((res_test+res_test_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_1_Selected_Strategies_ss.pkl', 'wb') as file:
    #         pickle.dump((ss_test+ss_test_imp), file)
    #     res_test4 = []
    #     res_test8 = []
    # if number_of_optimization_periods == 2:
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test2+res_test2_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test4+res_test4_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_res.pkl', 'wb') as file:
    #         pickle.dump((res_test+res_test_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_2_Selected_Strategies_ss.pkl', 'wb') as file:
    #         pickle.dump((ss_test+ss_test_imp),file)
    #     res_test8 = []
    # if number_of_optimization_periods == 3:
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_2_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test2+res_test2_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_4_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test4+res_test4_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_8_All_Strategies.pkl', 'wb') as file:
    #         pickle.dump((res_test8+res_test8_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_res.pkl', 'wb') as file:
    #         pickle.dump((res_test+res_test_imp), file)
    #     with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_3_Selected_Strategies_ss.pkl', 'wb') as file:
    #         pickle.dump((ss_test+ss_test_imp),file)

    # with open(f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl', 'wb') as file:
    #     pickle.dump((weights+[weights_imp[0][1]]), file)


****************************************************************************************************
LTFH
len2: 31
len4: 0
len8: 0
len_res_test: 31
len_ss_test: 31
Weights: 8
weights_supposed_to_be: 8
len2_imp: 1
len4_imp: 0
len8_imp: 0
len_res_test_imp: 1
len_ss_test_imp: 1
Weights_imp: 1
****************************************************************************************************


In [14]:
import eikon as ek
ek.set_app_key('57d4eb136d18406abbc3a8f17f2426f8de97d748')

In [15]:
import datetime
import datetime as dt
import dateutil.relativedelta
from datetime import datetime
from MCMC.MCMC import MCMC

In [21]:
def get_constituents(index):
    ic, err = ek.get_data(index, ['TR.IndexConstituentRIC'])
    lj, err = ek.get_data(index,
                          ['TR.IndexJLConstituentChangeDate',
                           'TR.IndexJLConstituentRIC.change',
                           'TR.IndexJLConstituentRIC'],
                          {'SDate': '0D', 'EDate': '-55M', 'IC': 'B'})
    lj['Date'] = pd.to_datetime(lj['Date']).dt.date
    lj.sort_values(['Date', 'Change'], ascending=False, inplace=True)
    dates = [dt.date(2007, 1, 30)]
    i = 0
    while (dates[0] + dateutil.relativedelta.relativedelta(months=+i + 1)) < dt.date.today():
        dates.append(dates[0] + dateutil.relativedelta.relativedelta(months=+i + 1))
        i = i + 1
    dates.append(dt.date.today())
    df = pd.DataFrame(index=dates, columns=['Index Constituents'])
    ic_list = ic['Constituent RIC'].tolist()
    for i in range(len(dates)):
        #print(str(dates[len(dates) - i - 1]))
        df.at[dates[len(dates) - i - 1], 'Index Constituents'] = ic_list[:]
        for j in lj.index:
            if lj['Date'].loc[j] <= dates[len(dates) - i - 1]:
                if lj['Date'].loc[j] > dates[len(dates) - i - 2]:
                    if lj['Change'].loc[j] == 'Joiner':
                        #print('Removing ' + lj['Constituent RIC'].loc[j])
                        ic_list.remove(lj['Constituent RIC'].loc[j])
                    elif lj['Change'].loc[j] == 'Leaver':
                        #print('Adding ' + lj['Constituent RIC'].loc[j])
                        ic_list.append(lj['Constituent RIC'].loc[j])
                else:
                    break
    try:
        try:
            df.index = pd.date_range(start=str(df.index[0])[:10], end=str(df.index[-1].replace(month=df.index[-1].month+1))[:10], freq=f'1M')
        except:
            df.index = pd.date_range(start=str(df.index[0])[:10],end=str(df.index[-1].replace(day = 30).replace(month=df.index[-1].month + 1))[:10], freq=f'1M')
    except:
        try:
            df.index = pd.date_range(start=str(df.index[0])[:10], end=str(df.index[-1].replace(month=1, year=df.index[-1].year + 1))[:10], freq=f'1M')
        except:
            df.index = pd.date_range(start=str(df.index[0])[:10],end=str(df.index[-1].replace(day = 30).replace(month=1, year=df.index[-1].year + 1))[:10], freq=f'1M')
    return df
def prepare_portfolio_data(tickers, recalibrating_months,api,country):
    def get_data(ticker, api,country):
        if api == "yfinance":

            temp_og = yf.download(ticker, start = '2007-07-01', end= str(date.today()+timedelta(1)))
            if len(temp_og)==0:
                temp_og = yf.download(ticker, start='2007-07-01', end=str(date.today()))
            temp_og.reset_index(inplace=True)
            temp_og.drop(['Adj Close'], axis=1, inplace=True)
            if ticker=="GOLDBEES.NS":
                temp_og = temp_og.loc[temp_og["Close"]>1]
            temp_og = add_fisher(temp_og)

        if api =="investpy":
            temp_og = get_data_investpy(symbol=ticker, country=country, from_date="01/07/2007",to_date=(date.today()+timedelta(1)).strftime("%d/%m/%Y"))
            temp_og.reset_index(inplace=True)
            temp_og = add_fisher(temp_og)

        if api == "reuters":
            temp_og = ek.get_timeseries(ticker, start_date='2007-07-01', end_date=str(date.today() + timedelta(1)))
            temp_og.reset_index(inplace=True)
            temp_og.rename(columns={"HIGH": "High", "CLOSE": "Close", "LOW": "Low", "OPEN": "Open", "VOLUME": "Volume"},
                           inplace=True)
            temp_og.drop(['COUNT'], axis=1, inplace=True)

        return temp_og

    data = pd.DataFrame()
    for ticker in tickers:
        try:
            temp_og = get_data(ticker, api,country)
            data = pd.concat([data, temp_og["Close"].to_frame().astype(float).rename(columns={"Close":ticker}).set_index(temp_og["Date"])], axis=1)
            data[f"{ticker}Return"] = np.log(data[ticker]/data[ticker].shift(1))
            data[f"{ticker}ROC0.5"] = data[ticker].pct_change(10)
            data[f"{ticker}ROC1"] = data[ticker].pct_change(21)
            data[f"{ticker}ROC3"] = data[ticker].pct_change(63)
            data[f"{ticker}ROC6"] = data[ticker].pct_change(126)
            data[f"{ticker}ROC9"] = data[ticker].pct_change(189)
            data[f"{ticker}ROC12"] = data[ticker].pct_change(252)
            data[f"{ticker}SD12"] = data[ticker].pct_change().rolling(252).std()
            data[f"{ticker}FReturn"] = data[ticker].shift(-recalibrating_months*21) / data[ticker] - 1
        except:
            print(f"{ticker} not processed")
    data.reset_index(inplace=True)
    return data
def get_weights_stocks_live(constituents, topn, test_monthsf, train_monthsf, datesf, temp_ogf, save=True):
    inputs = []
    for date_i in range(len(datesf) - (int(train_monthsf/test_monthsf) + 1)):
        inputs.append([temp_ogf, topn,datesf,date_i,train_monthsf,test_monthsf, constituents ])

    results = recalibrate_weights_stocks(inputs[-1])

    # res_test2 = [pd.DataFrame(columns=["Lookback", "Low Bound", "High Bound", "AvgWinLoss",  "Sortino", "Optimization_Years"])] * len(results2)
    res_test = [pd.DataFrame(columns=["Ticker", "WtROC0.5", "WtROC1", "WtROC3", "WtROC6", "WtROC9", "WtROC12", "Accelerating Momentum"])]
    res_test[0] = pd.concat(
        [res_test[0], results[1].reset_index().drop(['index'], axis=1)], axis=0)

    return res_test

def recalibrate_weights_stocks(inp):

    def alpha(*args):
        weights = pd.DataFrame(args)
        weights = weights / weights.sum()
        for ticker in tickers:
            df = data[[f"{ticker}ROC0.5", f"{ticker}ROC1", f"{ticker}ROC3", f"{ticker}ROC6", f"{ticker}ROC9",
                       f"{ticker}ROC12", f"{ticker}SD12", f"{ticker}FReturn"]]
            df[f"{ticker}AM"] = np.dot(df[[f"{ticker}ROC0.5", f"{ticker}ROC1", f"{ticker}ROC3", f"{ticker}ROC6",
                                           f"{ticker}ROC9", f"{ticker}ROC12"]], weights)
            data[f"{ticker}AM"] = df[f"{ticker}AM"] / df[f"{ticker}SD12"]
        return data[[f"{ticker}AM" for ticker in tickers]].to_numpy().ravel()

    def prior(params):
        if (params[0] < 0) | (params[0] > 1):
            return 0
        if (params[1] < 0) | (params[1] > 1):
            return 0
        if (params[2] < 0) | (params[2] > 1):
            return 0
        if (params[3] < 0) | (params[3] > 1):
            return 0
        if (params[4] < 0) | (params[4] > 1):
            return 0
        if (params[5] < 0) | (params[5] > 1):
            return 0
        return 1

    # Optimizing weights for entire portfolio
    temp_ogf = inp[0]
    top_n = inp[1]
    datesf = inp[2]
    date_i = inp[3]
    train_monthsf = inp[4]
    test_monthsf = inp[5]
    constituents = inp[6]

    # data_og = temp_ogf.loc[(temp_ogf["Date"] > str(datesf[date_i])) & (temp_ogf["Date"] < str(datesf[date_i + int(train_monthsf/test_monthsf) + int(test_monthsf/test_monthsf)]))].reset_index().drop(['index'], axis=1)

    # Adjustment made for forward returns
    data_og = temp_ogf.loc[(temp_ogf["Date"] > str(datesf[date_i])) & (
                temp_ogf["Date"] < str(datesf[date_i + int(train_monthsf / test_monthsf) - 1]))].reset_index().drop(
        ['index'], axis=1)

    tickers_in_index = constituents.loc[datesf[date_i]][0]

    #data = data_og.dropna(axis=1, how='all')
    #data = data.dropna()
    data = data_og.dropna(axis=1, how='any')

    tickers1 = []
    for column in data.columns[1:]:
        if column.endswith("Return") | column.endswith("FReturn") | column.endswith("ROC0.5") | column.endswith(
                "ROC1") | column.endswith("ROC3") | \
                column.endswith("ROC6") | column.endswith("ROC9") | column.endswith("ROC12") | column.endswith("SD12"):
            continue
        else:
            tickers1.append(column)

    tickers = []
    for ticker in tickers1:
        if ((f"{ticker}" in data.columns[1:]) & (f"{ticker}ROC0.5" in data.columns[1:]) & (
                f"{ticker}ROC1" in data.columns[1:]) & (
                f"{ticker}ROC3" in data.columns[1:]) & (f"{ticker}ROC6" in data.columns[1:]) &
                (f"{ticker}ROC9" in data.columns[1:]) & (f"{ticker}ROC12" in data.columns[1:]) & (
                        f"{ticker}SD12" in data.columns[1:]) & (f"{ticker}FReturn" in data.columns[1:])):
            #print(ticker)
            tickers.append(ticker)

    for ticker in tickers:
        if not ticker in tickers_in_index:
            tickers.remove(ticker)

    random_starts = 20
    iterations = 200
    guess_list = [np.random.dirichlet(np.ones(6), size=1).tolist()[0] for i in range(random_starts)]
    res = pd.DataFrame(columns=["WtROC0.5", "WtROC1", "WtROC3", "WtROC6", "WtROC9", "WtROC12", "NMIS"])

    try:
        for guess in guess_list:
            mc = MCMC(alpha_fn=alpha, alpha_fn_params_0=guess,
                      target=data[[f"{ticker}FReturn" for ticker in tickers]].to_numpy().ravel(), num_iters=iterations,
                      prior=prior, optimize_fn=None, lower_limit=0, upper_limit=1)
            rs = mc.optimize()
            res_iter = [{"WtROC0.5": mc.analyse_results(rs, top_n=iterations)[0][i][0],
                         "WtROC1": mc.analyse_results(rs, top_n=iterations)[0][i][1],
                         "WtROC3": mc.analyse_results(rs, top_n=iterations)[0][i][2],
                         "WtROC6": mc.analyse_results(rs, top_n=iterations)[0][i][3],
                         "WtROC9": mc.analyse_results(rs, top_n=iterations)[0][i][4],
                         "WtROC12": mc.analyse_results(rs, top_n=iterations)[0][i][5],
                         "NMIS": mc.analyse_results(rs, top_n=iterations)[1][i]} \
                        for i in range(iterations)]
            res_iter = pd.DataFrame(res_iter)
            res = pd.concat([res, res_iter], axis=0)
        res = res.sort_values("NMIS", axis=0, ascending=False).reset_index(drop=True)
        chosen_weights = pd.DataFrame(
            [res.iloc[0]["WtROC0.5"], res.iloc[0]["WtROC1"], res.iloc[0]["WtROC3"], res.iloc[0]["WtROC6"],
             res.iloc[0]["WtROC9"], res.iloc[0]["WtROC12"]])
        chosen_weights = chosen_weights / chosen_weights.sum()
        am = []
        for ticker in tickers:
            am.append({"Ticker": ticker, "WtROC0.5": float(chosen_weights.iloc[0]), "WtROC1": float(chosen_weights.iloc[1]),
                       "WtROC3": float(chosen_weights.iloc[2]), "WtROC6": float(chosen_weights.iloc[3]),
                       "WtROC9": float(chosen_weights.iloc[4]), "WtROC12": float(chosen_weights.iloc[5]),
                       "Accelerating Momentum": np.dot(
                           data_og.iloc[-1][[f"{ticker}ROC0.5", f"{ticker}ROC1", f"{ticker}ROC3",
                                             f"{ticker}ROC6", f"{ticker}ROC9", f"{ticker}ROC12"]],
                           chosen_weights)[0] / data_og.iloc[-1][f"{ticker}SD12"]})
        am = pd.DataFrame(am)
        am = am.sort_values("Accelerating Momentum", axis=0, ascending=False).reset_index(drop=True)
        am = am.iloc[:top_n]
    except:
        am = pd.DataFrame(columns = ["Ticker","WtROC0.5","WtROC1","WtROC3","WtROC6","WtROC9","WtROC12","Accelerating Momentum"])
    return date_i, am

In [20]:
#NASDAQ AM and Constituent rebalance

# #Recalibrating AM Constituents
print(f"Start Recalibrating constituents of NDX at {datetime.now()}")
index = ".NDX"
constituents = get_constituents(index)

with open(f'NDX_Constituents.pkl', 'wb') as file:
    pickle.dump(constituents, file)

print(f"End Recalibrating constituents of NDX at {datetime.now()}")


Start Recalibrating constituents of NDX at 2022-02-01 13:23:27.600175
    Instrument Constituent RIC
0         .NDX         GILD.OQ
1         .NDX         PAYX.OQ
2         .NDX          KHC.OQ
3         .NDX         BIDU.OQ
4         .NDX         BIIB.OQ
..         ...             ...
96        .NDX         MRNA.OQ
97        .NDX         ROST.OQ
98        .NDX         VRSK.OQ
99        .NDX         AMZN.OQ
100       .NDX         VRTX.OQ

[101 rows x 2 columns]
End Recalibrating constituents of NDX at 2022-02-01 13:23:34.847531


In [62]:
# #Recalibrating AM
print(f"Start Recalibrating AM of NDX at {datetime.now()}")
with open(f'NASDAQ_RecalibPeriod_{int(1)}.pkl', 'rb') as file:
    assets = pickle.load(file)
with open(f'NDX_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all + constituents.iloc[i][0]
tickers = list(set(constituents_all))
tickers = [ticker.partition('.')[0] for ticker in tickers]

# removing duplicate tickers
tickers_remove = ['DISCK', 'FOXA', 'GOOG', 'LBTYK', 'LILAK', 'TFCFA']
for remove in tickers_remove:
    tickers.remove(remove)
recalibrating_months = 1
top_nassets = 8
training_period = 24  # 24/48/96
dates_recalibrating = valid_dates(pd.date_range(start="2007-01-01", end="2024-06-15", freq=f'{recalibrating_months}M'))
data_inp = prepare_portfolio_data(tickers, recalibrating_months, "investpy", "united states")
assets.append(get_weights_stocks_live(constituents, top_nassets, recalibrating_months, training_period,dates_recalibrating, data_inp, save=False)[0])
with open(f'NASDAQ_RecalibPeriod_{int(1)}.pkl', 'wb') as file:
    pickle.dump(assets, file)
print(f"End Recalibrating AM of NDX at {datetime.now()}")


Start Recalibrating AM of NDX at 2022-02-01 13:48:32.597945


C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{ticker}Return"] = np.log(data[ticker]/data[ticker].shift(1))
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{ticker}ROC0.5"] = data[ticker].pct_change(10)
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

EFII not processed
RHT not processed
SHPG not processed
VIAB not processed
LVNTA not processed
MXIM not processed
ESRX not processed
ALXN not processed
TFCF not processed
CA not processed
CELG not processed


C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:86: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data.reset_index(inplace=True)
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{ticker}AM"] = np.dot(df[[f"{ticker}ROC0.5", f"{ticker}ROC1", f"{ticker}ROC3", f"{ticker}ROC6",
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

End Recalibrating AM of NDX at 2022-02-01 14:42:10.578969


In [19]:
#NIFTY AM and Constituent rebalance
# #Recalibrating AM Constituents
print(f"Start Recalibrating constituents of BSE at {datetime.now()}")
index = ".BSESN"
constituents = get_constituents(index)

with open(f'BSE_Constituents.pkl', 'wb') as file:
    pickle.dump(constituents, file)

print(f"End Recalibrating constituents of BSE at {datetime.now()}")

Start Recalibrating constituents of BSE at 2022-02-01 13:23:01.200342
        0  TR.INDEXCONSTITUENTRIC
0  .BSESN                    <NA>


KeyError: 'Constituent RIC'

In [43]:
with open(f'BSE_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)

In [38]:
constituents.loc[pd.to_datetime("2022-02-28"), "Index Constituents"] = constituents.loc["2022-01-31"]["Index Constituents"]


In [42]:
with open(f'BSE_Constituents.pkl', 'wb') as file:
    pickle.dump(constituents, file)

In [49]:
# #Recalibrating AM
with open(f'BSE_RecalibPeriod_{int(1)}.pkl', 'rb') as file:
    assets = pickle.load(file)
with open(f'BSE_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all + constituents.iloc[i][0]
tickers = list(set(constituents_all))
tickers = [ticker[:-3] for ticker in tickers]
# Restricted Stock
tickers.remove('AXBK')

recalibrating_months = 1
top_nassets = 10
training_period = 24  # 24/48/96
dates_recalibrating = valid_dates(
    pd.date_range(start="2009-01-01", end="2024-06-15", freq=f'{recalibrating_months}M'))
data_inp = prepare_portfolio_data(tickers, recalibrating_months, "investpy", "india")

data_inp = data_inp[data_inp['Date'] != '2020-11-14']
data_inp = data_inp[data_inp['Date'] != '2020-11-17']
assets.append(get_weights_stocks_live(constituents, top_nassets, recalibrating_months, training_period,dates_recalibrating, data_inp, save=False)[0])
with open(f'BSE_RecalibPeriod_{int(1)}.pkl', 'wb') as file:
    pickle.dump(assets, file)
print(f"End Recalibrating AM of BSE at {datetime.now()}")


C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{ticker}Return"] = np.log(data[ticker]/data[ticker].shift(1))
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{ticker}ROC0.5"] = data[ticker].pct_change(10)
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

End Recalibrating AM of BSE at 2022-02-01 13:41:09.271541


In [52]:
#MIDCAP AM and Constituent rebalance

# #Recalibrating AM Constituents
print(f"Start Recalibrating constituents of Midcap at {datetime.now()}")

tickers = ['ESCO', 'MINT', 'BLKI', 'BAJE', 'COFO', 'VODA', 'SRFL', 'MAXI', 'CCRI', 'MNFL', 'TOPO',\
           'VOLT', 'CNBK', 'GMRI', 'BHEL', 'AUFI', 'GODR', \
           'ZEE', 'TTPW', 'ASOK', 'SRTR', 'BATA', 'SUTV', 'PAGE', 'PWFC', 'EXID', 'MGAS', 'CHLA',
           'TVSM', 'UNBK', 'BOB', 'CUMM', 'TRCE', 'IDFB', 'CAST', 'FED', 'GLEN', \
           'LICH', 'RECM', 'MMFS', 'INBF', 'AMAR', 'BOI', 'INIR', 'LTFH', 'APLO', 'SAIL', 'RATB',\
           'JNSP', 'BFRG']
index = ".NIMDCP50"
constituents = get_constituents(index)
for i in range(len(constituents)):
    constituents.iloc[i]["Index Constituents"] = tickers

with open(f'MIDCAP_CSV_Constituents.pkl', 'wb') as file:
    pickle.dump(constituents, file)

print(f"End Recalibrating constituents of Nifty Midcap at {datetime.now()}")

Start Recalibrating constituents of Midcap at 2022-02-01 13:43:05.445382


2022-02-01 13:43:07,498 P[11236] [MainThread 3988] Eikon Proxy not running or cannot be reached. Please read the documentation on troubleshooting


EikonError: Error code 401 | Eikon Proxy not running or cannot be reached. Please read the documentation on troubleshooting

In [53]:
with open(f'MIDCAP_CSV_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)


In [55]:
constituents.loc[pd.to_datetime("2022-02-28"), "Index Constituents"] = constituents.loc["2022-01-31"]["Index Constituents"]


In [56]:
constituents

,Index Constituents
2007-01-31,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
2007-02-28,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
2007-03-31,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
2007-04-30,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
2007-05-31,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
...,...
2021-10-31,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
2021-11-30,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
2021-12-31,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."
2022-01-31,"[ESCO, MINT, BLKI, BAJE, COFO, VODA, SRFL, MAX..."


In [57]:
with open(f'MIDCAP_CSV_Constituents.pkl', 'wb') as file:
    pickle.dump(constituents, file)

In [58]:
# #Recalibrating AM
index = ".NIMDCP50"
with open(f'MIDCAP_CSV_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all + constituents.iloc[i][0]
tickers = list(set(constituents_all))
top_nassets = 8
recalibrating_months = 1
training_period = 24  # 24/48/96
dates_recalibrating = valid_dates(
    pd.date_range(start="2007-01-01", end="2024-06-15", freq=f'{recalibrating_months}M'))
data_inp = prepare_portfolio_data(tickers, recalibrating_months, "investpy", "india")

data_inp = data_inp[(data_inp["Date"] < "2014-02-28") | (data_inp["Date"] > "2014-03-13")]
data_inp = data_inp[data_inp["Date"] != "2020-11-14"]
data_inp = data_inp[data_inp["Date"] != "2020-11-17"]

with open(f'MIDCAP_RecalibPeriod_{int(1)}.pkl', 'rb') as file:
    assets = pickle.load(file)
print(f"Start Recalibrating AM of Nifty Midcap at {datetime.now()}")
assets.append(get_weights_stocks_live(constituents, top_nassets, recalibrating_months, training_period,dates_recalibrating, data_inp, save=False)[0])
with open(f'MIDCAP_RecalibPeriod_{int(1)}.pkl', 'wb') as file:
    pickle.dump(assets, file)
print(f"End Recalibrating AM of Nifty Midcap at {datetime.now()}")

C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{ticker}Return"] = np.log(data[ticker]/data[ticker].shift(1))
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{ticker}ROC0.5"] = data[ticker].pct_change(10)
C:\Users\SUPRAB~1\AppData\Local\Temp/ipykernel_11236/3966994816.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

Start Recalibrating AM of Nifty Midcap at 2022-02-01 13:47:07.167866
End Recalibrating AM of Nifty Midcap at 2022-02-01 13:47:39.187547
